<a href="https://colab.research.google.com/github/Jollyhrothgar/deep_learning/blob/master/Lesson_3_The_Fashion_MNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

The Fashion MNIST dataset contains 70,000  images containing 10 different classes of clohting. Each clothing item is represented as a well-centered 28 by 28 grescale picture.

## Data Preparation

Each image is converted to a vector by flattening the image from a 28 $\times$ 28 to a 784 element vector.

The input layer will transform the vector into a densly connected 128 neuron output. This means that we multiply the 784 element vector representing an image with a 784 $\times$ 128 matrix to generate a new transformed vector representing the embedding.

For the activation function for each number, we use the relu function, which is useful in this case. ReLU has some interesting properties which make it useful here:

* When the weighted sum of the neuron is less than 0, we set this to zero, effectively de-activating any subsequent weight associated with this neuron.

There are lots of activation functions, and the properties of these functions tend to be designed to facilitate a few different things:

* Computation constraint when calculating the gradient during back propagation / loss minimization
* Try to solve the "vanishing gradient" problem - e.g., issues that create an ambiguous loss surface.
* Avoid "dead" neurons - e.g. neurons for whom the learned weight is zero

Activation functions can also be used map the final layer of the problem to classification or regression. ReLU should not be used for the final layer of a network - [more information here](https://www.analyticsvidhya.com/blog/2020/01/fundamentals-deep-learning-activation-functions-when-to-use-them/#:~:text=The%20ReLU%20function%20is%20another,neurons%20at%20the%20same%20time.)

Output layer for classification should be dense, and have the same number of neurons as there are classes. Additionally, the layer must use softmax activation, which converts the output to a probability.

# Load Libraries!

In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import tensorflow_datasets as tfds

tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt

# pretty progress bar
import tqdm
import tqdm.auto
tqdm.tqdm = tqdm.auto.tqdm

print(tf.__version__)

ModuleNotFoundError: ignored